## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [47]:
import mysql.connector

def get_mysql_connection():
    return mysql.connector.connect(
        host="localhost",
        user="isapereira",
        password="12345"
    )

with get_mysql_connection() as cnx:
    cursor = cnx.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")
    print("Banco criado/verificado com sucesso")


Banco criado/verificado com sucesso


In [48]:
with get_mysql_connection() as cnx:
    cursor = cnx.cursor()
    cursor.execute("SHOW DATABASES;")

    for db in cursor:
        print(db)


('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


In [49]:
import pandas as pd

df_eletronicos = pd.read_csv("/home/isabela/pipeline-python-mongo-mysql/avaliacao/tb_4_5_em_diante.csv")
df_eletronicos.head()


,_id,id,Categoria,Descrição,Imagem,Preço,Título,Nota,Total_Avaliacoes
0,695415213500e8a7f25a62de,9.0,electronics,USB 3.0 and USB 2.0 Compatibility Fast data tr...,https://fakestoreapi.com/img/61IBBVJvSDL._AC_S...,64.0,WD 2TB Elements Portable External Hard Drive -...,4.1,259.0
1,695415213500e8a7f25a62df,10.0,electronics,"Easy upgrade for faster boot up, shutdown, app...",https://fakestoreapi.com/img/61U7T1koQqL._AC_S...,109.0,SanDisk SSD PLUS 1TB Internal SSD - SATA III 6...,4.7,500.0
2,695415213500e8a7f25a62e0,11.0,electronics,3D NAND flash are applied to deliver high tran...,https://fakestoreapi.com/img/71kWymZ+c+L._AC_S...,109.0,Silicon Power 256GB SSD 3D NAND A55 SLC Cache ...,4.6,400.0
3,695415213500e8a7f25a62e1,12.0,electronics,"Expand your PS4 gaming experience, Play anywhe...",https://fakestoreapi.com/img/61mtL65D4cL._AC_S...,114.0,WD 4TB Gaming Drive Works with Playstation 4 P...,4.8,319.0
4,695415213500e8a7f25a62e2,13.0,electronics,21. 5 inches Full HD (1920 x 1080) widescreen ...,https://fakestoreapi.com/img/81QpkIctqPL._AC_S...,599.0,Acer SB220Q bi 21.5 inches Full HD (1920 x 108...,4.8,400.0


In [50]:
df_eletronicos.columns

Index(['_id', 'id', 'Categoria', 'Descrição', 'Imagem', 'Preço', 'Título',
       'Nota', 'Total_Avaliacoes'],
      dtype='object')

In [51]:
df_eletronicos.shape

(7, 9)

In [52]:
import mysql.connector

# 1. Reestabeleça a conexão (ajuste seus dados se necessário)
try:
    cnx = mysql.connector.connect(
        host="localhost",
        user="isapereira",
        password="12345"
    )
    

    if cnx.is_connected():
        cursor = cnx.cursor()
        
        # 2. Garanta que o banco de dados existe
        cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")
        
        # 3. Execute a criação da tabela
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS dbprodutos.tb_eletronicos(
                id VARCHAR(100),
                Titulo VARCHAR(255),
                Preco FLOAT(10,2),
                Nota FLOAT(10,2),
                Total_Avaliacoes INT,
                PRIMARY KEY (id)
            );
        """)
        
        print("Tabela 'tb_eletronicos' verificada/criada com sucesso!")
        
        # 4. Verifique se a tabela aparece
        cursor.execute("USE dbprodutos;")
        cursor.execute("SHOW TABLES;")
        for tb in cursor:
            print(f"Tabela encontrada: {tb}")

        cursor.close()

except mysql.connector.Error as err:
    print(f"Erro ao conectar ou executar: {err}")

finally:
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()

Tabela 'tb_eletronicos' verificada/criada com sucesso!
Tabela encontrada: ('tb_eletronicos',)


In [57]:
import pandas as pd
import numpy as np
import mysql.connector

# 1. Carregue o arquivo APENAS UMA VEZ antes de começar o processo
df_final = pd.read_csv('../avaliacao/tb_4_5_em_diante.csv')

# 2. Substitua os NaNs por None (para o MySQL entender como NULL)
df_final = df_final.replace({np.nan: None})

# 3. Prepare o comando SQL
comando_insert = """
    INSERT INTO dbprodutos.tb_eletronicos (id, Titulo, Preco, Nota, Total_Avaliacoes)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
    Titulo=VALUES(Titulo), Preco=VALUES(Preco), Nota=VALUES(Nota), Total_Avaliacoes=VALUES(Total_Avaliacoes);
"""

try:
    # 4. Estabeleça a conexão
    cnx = mysql.connector.connect(host="localhost", user="isapereira", password="12345")
    cursor = cnx.cursor()

    print("Iniciando a inserção dos dados...")

    # 5. Percorra o DataFrame limpo
    for i, linha in df_final.iterrows():
        # Tratamento individual para garantir tipos corretos sem quebrar no 'None'
        # Se for None, não tentamos converter para float/int
        dados = (
            str(linha['id']), 
            str(linha['Título']), 
            float(linha['Preço']) if linha['Preço'] is not None else None,
            float(linha['Nota']) if linha['Nota'] is not None else None,
            int(linha['Total_Avaliacoes']) if linha['Total_Avaliacoes'] is not None else 0
        )
        cursor.execute(comando_insert, dados)

    # 6. Efetue o commit para salvar no banco
    cnx.commit()
    
    # 7. Verificação final
    cursor.execute("SELECT COUNT(*) FROM dbprodutos.tb_eletronicos;")
    total = cursor.fetchone()[0]
    print(f"Sucesso! {len(df_final)} linhas processadas. Total no banco: {total}")

    cursor.close()

except mysql.connector.Error as err:
    print(f"Erro no MySQL: {err}")
finally:
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()

Iniciando a inserção dos dados...
Sucesso! 7 linhas processadas. Total no banco: 7


In [59]:
import mysql.connector

try:
    # 1. Abre a conexão novamente
    cnx = mysql.connector.connect(
        host="localhost",
        user="isapereira",
        password="12345"
    )

    if cnx.is_connected():
        cursor = cnx.cursor()
        
        # 2. Seleciona o banco e lista as tabelas
        cursor.execute("USE dbprodutos;")
        cursor.execute("SHOW TABLES;")
        
        print("Conexão reestabelecida!")
        print("Tabelas encontradas no banco 'dbprodutos':")
        print("-" * 35)

        # 3. Itera sobre os resultados
        for (table_name,) in cursor:
            print(f"-> {table_name}")

        cursor.close()

except mysql.connector.Error as err:
    print(f"Erro ao tentar confirmar: {err}")

finally:
    # 4. Mantemos a boa prática de fechar ao terminar
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()

Conexão reestabelecida!
Tabelas encontradas no banco 'dbprodutos':
-----------------------------------
-> tb_eletronicos
